# Scrapping and preparing data

In [13]:
import pandas as pd
import tweepy as tw
import pickle as pkl
import getpass

## Kraggle data
The dataset from Kraggle _Jair Bolsonaro Twitter Data_ presents a collection of 3 thousands of tweets from president Jair Bolsonaro.

In [14]:
kraggle_data = pd.read_csv("bolsonaro_tweets.csv")
kraggle_data.date = pd.to_datetime(kraggle_data.date)
kraggle_data.head()

,date,text,likes,retweets,link
0,2020-04-29,O @govbr utilizará R$ 500 milhões para a aquis...,4250,808,https://twitter.com/jairbolsonaro/status/12555...
1,2020-04-29,"13- Mais de 17,7 mil brasileiros repatriados;\...",12314,1830,https://twitter.com/jairbolsonaro/status/12554...
2,2020-04-29,"10- 30,1 mil brasileiros curados da covid-19;...",13232,1854,https://twitter.com/jairbolsonaro/status/12554...
3,2020-04-29,"7- Suporte psicológico para equipes do SUS, v...",5121,941,https://twitter.com/jairbolsonaro/status/12554...
4,2020-04-29,4- Conclusão do Hospital de Campanha do Govern...,4247,882,https://twitter.com/jairbolsonaro/status/12554...


In [15]:
print("The time period of " +  str(len(kraggle_data))+" tweets are from " +  str(kraggle_data.date.min())[0:10] +" to " +  str(kraggle_data.date.max())[0:10] + ".")

The time period of 3120 tweets are from 2010-04-01 to 2020-04-29.


## Twitter scrapping

In [3]:
consumer_key = getpass.getpass()

········


In [4]:
consumer_secret = getpass.getpass()

········


In [5]:
access_token = getpass.getpass()

········


In [6]:
access_token_secret = getpass.getpass()

········


In [7]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [9]:
tweets_list = [tweet for tweet in tw.Cursor(api.user_timeline,id='jairbolsonaro', tweet_model = 'extended').items()]

In [17]:
print("Oldest tweet is from " + str(tweets_list[-1].created_at) + ".")

Oldest tweet is from 2019-05-16 17:22:36.
